In [51]:
import json
import requests
import datetime as dt
import pickle
import pandas as pd

# An api key is emailed to you when you sign up to a plan
api_key = 'da3195c8356b3b0ee3e5a6682ed9c537'


In [52]:
def get_odds_data():
    sports_response = requests.get('https://api.the-odds-api.com/v3/odds', params={
        'api_key': api_key,
        'sport': 'basketball_ncaab',
        'region': 'us',
        'oddsFormat': 'american'
    })


    sports_json = json.loads(sports_response.text)
    date = dt.datetime.today().date().strftime('%Y-%m-%d')
    with open(str(date)+'.json', 'w') as outfile:
        json.dump(sports_json, outfile)
    

In [53]:
get_odds_data()

In [54]:
date = dt.datetime.today().date().strftime('%Y-%m-%d')
with open(str(date)+'.json') as f:
    sports_json = json.load(f)

In [55]:
def getTeamData(name):
    name = name.replace(" ", "%20")
    baseUrl="https://api.fastbreakstats.com/teams/"
    params="?q=current"
    url=baseUrl+name+params

    r = requests.get(url = url).json()[0]
    bpival = r['BPIval']
    kpval = r['KenPomVal']
    srsval = r['SRS']
    return kpval, bpival, srsval

In [56]:
def get_needed_odds1(conf):
    payout = round(((.1+(1-conf))/conf),4)
    if 100/payout > 100:
        odds = round(-100/payout)
    else:
        odds = round(100*payout)
    return odds

In [57]:
def get_ev1(odds,conf):
    if int(odds) > 0:
        payout = odds/100
    else:
        payout = -100/odds
    ev = (conf * payout) - (1-conf)
    return ev

In [58]:
def print_games():
    teams = pd.read_csv('teams_table.csv')
    loaded_model = pickle.load(open('6yr_LR.sav', 'rb'))
    for game in sports_json['data']:
        #Loop through and get odds
        odds = ['n/a','n/a']
        for site in game['sites']:
            if site['site_key'] == 'bovada':
                odds = site['odds']['h2h']
                sitex = "bovada"
                break
            if site['site_key'] == 'draftkings':
                odds = site['odds']['h2h']
                sitex = "DK"
            if site['site_key'] == 'williamhill_us':
                odds = site['odds']['h2h']
                sitex = "WH"
            if site['site_key'] == 'fanduel':
                odds = site['odds']['h2h']
                sitex = "FD"

        if odds == ['n/a','n/a']:
            print("There are no odds for {0}".format(game['teams']))
            print()
            continue

        home = game['home_team']
        if game['home_team'] == game['teams'][0]:
            away = game['teams'][1]
            home_odds = odds[0]
            away_odds = odds[1]
        else:
            away = game['teams'][0]
            home_odds = odds[1]
            away_odds = odds[0]

        try:
            home = home.replace("-", " ")
            home = home.replace("(", "")
            home = home.replace(")", "")
            home = home.replace(".", "")
            away = away.replace("-", " ")
            away = away.replace("(", "")
            away = away.replace(")", "")
            away = away.replace(".", "")
            homekp, homebpi, homesrs = getTeamData(teams.loc[teams['odds_api_name'] == home]['kenpom_name'].values[0])
            awaykp, awaybpi, awaysrs = getTeamData(teams.loc[teams['odds_api_name'] == away]['kenpom_name'].values[0])
        except Exception as e:
            print(e)
            print(game['teams'])
            print()
            continue

        res = loaded_model.predict_proba([[float(awaykp), float(awaybpi), float(awaysrs), float(homekp), float(homebpi), float(homesrs)]])
    #     print(res[0])
        print("{0} @ {1}".format(away,home))
        away_needed_odds = get_needed_odds1(res[0][0])    
        home_needed_odds = get_needed_odds1(res[0][1])
        away_ev = get_ev1(away_odds,res[0][0])
        home_ev = get_ev1(home_odds,res[0][1])
        if away_odds > away_needed_odds:
            print("{0} @ {1} on {2}".format(away,home,sitex))
            print("{0} needs {1} and has {2} odds with an Expected Value of {3}".format(away,away_needed_odds,away_odds,away_ev))
#             print()
        if home_odds > home_needed_odds:
            print("{0} @ {1} on {2}".format(away,home,sitex))
            print("{0} needs {1} and has {2} odds with an Expected Value of {3}".format(home,home_needed_odds,home_odds,home_ev))
        print()

In [59]:
print_games()

Delaware St Hornets @ UNC Wilmington Seahawks

There are no odds for ['NJIT Highlanders', 'Rider Broncs']

Saint Peter's Peacocks @ St Francis BKN Terriers

Xavier Musketeers @ Creighton Bluejays

UCLA Bruins @ Oregon Ducks
UCLA Bruins @ Oregon Ducks on DK
Oregon Ducks needs -186.0 and has -143 odds with an Expected Value of 0.21512311606007084

Towson Tigers @ George Mason Patriots
Towson Tigers @ George Mason Patriots on bovada
George Mason Patriots needs -324.0 and has -210 odds with an Expected Value of 0.24087092833030116

SE Louisiana Lions @ Grambling St Tigers

Idaho State Bengals @ N Colorado Bears

Rutgers Scarlet Knights @ Ohio State Buckeyes

Georgetown Hoyas @ Seton Hall Pirates

South Carolina Upstate Spartans @ Tennessee Volunteers

Northwestern St Demons @ Washington St Cougars

Providence Friars @ Butler Bulldogs

CSU Bakersfield Roadrunners @ Pepperdine Waves
CSU Bakersfield Roadrunners @ Pepperdine Waves on bovada
Pepperdine Waves needs -330.0 and has -300 odds with 

In [42]:
sports_json['data'][0]

{'sport_key': 'basketball_ncaab',
 'sport_nice': 'NCAAB',
 'teams': ['Delaware St Hornets', 'UNC Wilmington Seahawks'],
 'commence_time': 1608742800,
 'home_team': 'UNC Wilmington Seahawks',
 'sites': [{'site_key': 'fanduel',
   'site_nice': 'FanDuel',
   'last_update': 1608733229,
   'odds': {'h2h': [1600, -4500]}},
  {'site_key': 'bovada',
   'site_nice': 'Bovada',
   'last_update': 1608733252,
   'odds': {'h2h': [1175, -3448]}},
  {'site_key': 'betrivers',
   'site_nice': 'BetRivers',
   'last_update': 1608733259,
   'odds': {'h2h': [1600, -10000]}},
  {'site_key': 'sugarhouse',
   'site_nice': 'SugarHouse',
   'last_update': 1608733230,
   'odds': {'h2h': [1600, -10000]}},
  {'site_key': 'draftkings',
   'site_nice': 'DraftKings',
   'last_update': 1608733229,
   'odds': {'h2h': [1600, -10000]}},
  {'site_key': 'betmgm',
   'site_nice': 'BetMGM',
   'last_update': 1608733259,
   'odds': {'h2h': [1850, -10000]}}],
 'sites_count': 6}

In [44]:
dt.datetime.today().date()

datetime.date(2020, 12, 23)

In [45]:
dt.datetime.fromtimestamp(1608667200).date() == dt.datetime.now().date()

False

In [46]:
def get_bet_games():
    teams = pd.read_csv('teams_table.csv')
    loaded_model = pickle.load(open('6yr_LR.sav', 'rb'))
    games_to_bet = []
    for game in sports_json['data']:
        #Loop through and get odds
        odds = ['n/a','n/a']
        if game['commence_time'] < dt.datetime.now().timestamp():
            continue
        if dt.datetime.fromtimestamp(game['commence_time']).date() > dt.datetime.now().date():
            continue
        for site in game['sites']:
            if site['site_key'] == 'bovada':
                odds = site['odds']['h2h']
                sitex = "bovada"
                break
            if site['site_key'] == 'draftkings':
                odds = site['odds']['h2h']
                sitex = "DK"
            if site['site_key'] == 'williamhill_us':
                odds = site['odds']['h2h']
                sitex = "WH"
            if site['site_key'] == 'fanduel':
                odds = site['odds']['h2h']
                sitex = "FD"

        if odds == ['n/a','n/a']:
    #         print("There are no odds for {0}".format(game['teams']))
            continue

        home = game['home_team']
        if game['home_team'] == game['teams'][0]:
            away = game['teams'][1]
            home_odds = odds[0]
            away_odds = odds[1]
        else:
            away = game['teams'][0]
            home_odds = odds[1]
            away_odds = odds[0]

        try:
            home = home.replace("-", " ")
            home = home.replace("(", "")
            home = home.replace(")", "")
            home = home.replace(".", "")
            away = away.replace("-", " ")
            away = away.replace("(", "")
            away = away.replace(")", "")
            away = away.replace(".", "")
            homekp, homebpi, homesrs = getTeamData(teams.loc[teams['odds_api_name'] == home]['kenpom_name'].values[0])
            awaykp, awaybpi, awaysrs = getTeamData(teams.loc[teams['odds_api_name'] == away]['kenpom_name'].values[0])
        except Exception as e:
            print(e)
            print(game['teams'])
            pass

        res = loaded_model.predict_proba([[float(awaykp), float(awaybpi), float(awaysrs), float(homekp), float(homebpi), float(homesrs)]])
        away_needed_odds = get_needed_odds1(res[0][0])    
        home_needed_odds = get_needed_odds1(res[0][1])
        away_ev = get_ev1(away_odds,res[0][0])
        home_ev = get_ev1(home_odds,res[0][1])
        if away_odds > away_needed_odds:
            game_to_bet = {}
            game_to_bet['game'] = "{0} @ {1}".format(away,home)
            game_to_bet['starttime'] = dt.datetime.fromtimestamp(game['commence_time']).strftime('%Y-%m-%d %H:%M:%S CST')
            game_to_bet['betsite'] = sitex
            game_to_bet['team'] = away
            game_to_bet['odds_found'] = away_odds
            game_to_bet['odds_needed'] = away_needed_odds
            game_to_bet['ev'] = away_ev
            games_to_bet.append(game_to_bet)
        if home_odds > home_needed_odds:
            game_to_bet = {}
            game_to_bet['game'] = "{0} @ {1}".format(away,home)
            game_to_bet['starttime'] = dt.datetime.fromtimestamp(game['commence_time']).strftime('%Y-%m-%d %H:%M:%S CST')
            game_to_bet['betsite'] = sitex
            game_to_bet['team'] = home
            game_to_bet['odds_found'] = home_odds
            game_to_bet['odds_needed'] = home_needed_odds
            game_to_bet['ev'] = home_ev
            games_to_bet.append(game_to_bet)
    return games_to_bet

In [47]:
today = get_bet_games()

index 0 is out of bounds for axis 0 with size 0
['Illinois Fighting Illini', 'Penn State Nittany Lions']


In [48]:
print(today)

[{'game': 'NJIT Highlanders @ Rider Broncs', 'starttime': '2020-12-23 13:00:00 CST', 'betsite': 'bovada', 'team': 'NJIT Highlanders', 'odds_found': 100, 'odds_needed': -149.0, 'ev': 0.3168210806435736}, {'game': 'UCLA Bruins @ Oregon Ducks', 'starttime': '2020-12-23 14:00:00 CST', 'betsite': 'DK', 'team': 'Oregon Ducks', 'odds_found': -143, 'odds_needed': -186.0, 'ev': 0.21512311606007084}, {'game': 'Towson Tigers @ George Mason Patriots', 'starttime': '2020-12-23 15:00:00 CST', 'betsite': 'bovada', 'team': 'George Mason Patriots', 'odds_found': -210, 'odds_needed': -324.0, 'ev': 0.24087092833030116}, {'game': 'Illinois Fighting Illini @ Penn State Nittany Lions', 'starttime': '2020-12-23 17:30:00 CST', 'betsite': 'bovada', 'team': 'Penn State Nittany Lions', 'odds_found': 160, 'odds_needed': -330.0, 'ev': 1.195052081404999}, {'game': 'Georgia Tech Yellow Jackets @ UAB Blazers', 'starttime': '2020-12-23 19:00:00 CST', 'betsite': 'FD', 'team': 'UAB Blazers', 'odds_found': 102, 'odds_nee